<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/ABSA_pretrained_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Activate GPU and Install Dependencies

In [1]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
# Install required librariescol
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 47.3 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 45.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.0 MB/s 
     |████████████████████████████████| 134 kB 41.1 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 6.8 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 47.5 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 50.9 MB/s 
     |████████████████████████████████| 144 kB 43.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no long

#2. Preprocess data

In [3]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Create a smaller training dataset for faster training times
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}
{'text': "<br /><br />When I unsuspectedly rented A Thousand Acres, I thought I was in for an entertaining King Lear story and of course Michelle Pfeiffer was in it, so what could go wrong?<br /><br />Very quickly, 

In [5]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [6]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [17]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

In [18]:
# Define the evaluation metrics 
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [19]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [20]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/finetuning-sentiment-model-3000-samples is already a clone of https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
# Train the model
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 376


Step,Training Loss


Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-188
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-188/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-188/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/checkpoint-188/special_tokens_map.json
tokenizer config file saved in finetuning-sentiment-model-3000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/special_tokens_map.json
Saving model checkpoint to finetuning-sentiment-model-3000-samples/checkpoint-376
Configuration saved in finetuning-sentiment-model-3000-samples/checkpoint-376/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/checkpoint-376/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000

TrainOutput(global_step=376, training_loss=0.43361302639575716, metrics={'train_runtime': 534.152, 'train_samples_per_second': 11.233, 'train_steps_per_second': 0.704, 'total_flos': 783748564915968.0, 'train_loss': 0.43361302639575716, 'epoch': 2.0})

In [14]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16


Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

{'epoch': 2.0,
 'eval_accuracy': 0.8733333333333333,
 'eval_f1': 0.879746835443038,
 'eval_loss': 0.32088759541511536,
 'eval_runtime': 10.3589,
 'eval_samples_per_second': 28.96,
 'eval_steps_per_second': 1.834}

# 4. Analyzing new data with the model

In [15]:
# Upload the model to the Hub
trainer.push_to_hub()

Saving model checkpoint to finetuning-sentiment-model-3000-samples
Configuration saved in finetuning-sentiment-model-3000-samples/config.json
Model weights saved in finetuning-sentiment-model-3000-samples/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-3000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-3000-samples/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.36k/255M [00:00<?, ?B/s]

Upload file runs/Feb25_15-01-55_9b0859b6dbcf/events.out.tfevents.1645802065.9b0859b6dbcf.82.2: 100%|##########…

Upload file runs/Feb25_15-01-55_9b0859b6dbcf/events.out.tfevents.1645801429.9b0859b6dbcf.82.0:  96%|#########5…

To https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples
   c2aa231..0b88671  main -> main

To https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples
   0b88671..5cf9bbe  main -> main



'https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/commit/0b8867111d5afff659f36a25d29441e632329e0a'

In [16]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="saptarshidatta96/finetuning-sentiment-model-3000-samples")

sentiment_model(["I love this move", "This movie sucks!"])

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmps0_j9tkn


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

storing https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/401a5fc5bcb04ccd5db51a1a037916c0499da330391b1414b1ce0e5e23e39926.b397ca857184a9871736161bdc5bd2d66eceec5a2e6659984ded986a423b11af
creating metadata file for /root/.cache/huggingface/transformers/401a5fc5bcb04ccd5db51a1a037916c0499da330391b1414b1ce0e5e23e39926.b397ca857184a9871736161bdc5bd2d66eceec5a2e6659984ded986a423b11af
loading configuration file https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/401a5fc5bcb04ccd5db51a1a037916c0499da330391b1414b1ce0e5e23e39926.b397ca857184a9871736161bdc5bd2d66eceec5a2e6659984ded986a423b11af
Model config DistilBertConfig {
  "_name_or_path": "saptarshidatta96/finetuning-sentiment-model-3000-samples",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"


Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

storing https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/9f6f03c5bcdde81025bb6037b09be0475c1247f013e19abb3ff7afd90443a62f.dc959811ea11b2d6187ac0420d44495e87ec17ecbb8e7f7a767d38155d6b95e9
creating metadata file for /root/.cache/huggingface/transformers/9f6f03c5bcdde81025bb6037b09be0475c1247f013e19abb3ff7afd90443a62f.dc959811ea11b2d6187ac0420d44495e87ec17ecbb8e7f7a767d38155d6b95e9
https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpf2on9gni


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/dcf8ea346f469a02523bfe1e05b8a47e0ae9e0aa6000548c2727047c81101484.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/dcf8ea346f469a02523bfe1e05b8a47e0ae9e0aa6000548c2727047c81101484.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/d8e5f4a55912a8174bade7fb726cb8570559fd0763d38fe0cc2080f07eb1c31c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/saptarshidatta96/finetuning-sentiment-model-3000-samples/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/9f6f03c5bcdde81025bb6037b09be

[{'label': 'LABEL_1', 'score': 0.9587340950965881},
 {'label': 'LABEL_0', 'score': 0.945106565952301}]

In [26]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    """After requesting to be seated at an empty table, the waitress (who was so terribly burdened when we asked to move from the bar to the table to have dinner) asked us to get up from our seats and wait for a smaller table because a party of 3 just walked in.""",
    candidate_labels=['food', 'place', 'staff', 'miscellaneous', 'service', 'price', 'menu', 'ambience'],
)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
loading configuration file https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-mnli",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_la

{'labels': ['staff',
  'service',
  'place',
  'food',
  'miscellaneous',
  'menu',
  'ambience',
  'price'],
 'scores': [0.4780351221561432,
  0.31467998027801514,
  0.12634383141994476,
  0.02477204240858555,
  0.023725703358650208,
  0.014655341394245625,
  0.013461994007229805,
  0.004325991030782461],
 'sequence': 'After requesting to be seated at an empty table, the waitress (who was so terribly burdened when we asked to move from the bar to the table to have dinner) asked us to get up from our seats and wait for a smaller table because a party of 3 just walked in.'}